### GWAS POC

In [1]:
import xarray as xr
import zarr
import gcsfs
import pandas as pd
import numpy as np
import sgkit as sg
%load_ext autoreload
%autoreload 2

In [2]:
fs = gcsfs.GCSFileSystem()
store = gcsfs.mapping.GCSMap('rs-ukb/prep-data/gt-imputation/ukb_chr21.zarr', gcs=fs, check=True, create=False)
ds = xr.open_zarr(store, consolidated=True)
ds

,Array,Chunk
Bytes,2.46 TB,536.87 MB
Shape,"(1261158, 487409)","(1024, 131072)"
Count,4929 Tasks,4928 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,614.70 GB,134.22 MB
Shape,"(1261158, 487409)","(1024, 131072)"
Count,4929 Tasks,4928 Chunks
Type,bool,numpy.ndarray
,Array,Chunk


In [11]:
ds['variant_id'].values[:10]

array(['21:9411239_G_A', '21:9411245_C_A', '21:9411264_A_C',
       '21:9411267_G_T', '21:9411298_G_A', '21:9411302_G_T',
       '21:9411313_G_A', '21:9411332_G_T', '21:9411347_G_C',
       '21:9411354_A_G'], dtype=object)

In [12]:
ds['variant_rsid'].values[:10]

array(['rs559462325', 'rs181691356', 'rs548263598', 'rs561987868',
       '21:9411298_G_A', 'rs531010746', 'rs550852792', 'rs571137411',
       'rs539713234', '21:9411354_A_G'], dtype=object)

In [4]:
cols = {
    "x22007_0_0": 'plate',
    "x22008_0_0": 'well'
}
df_sid = pd.read_csv('gs://rs-ukb/prep-data/main/ukb_sample_qc.csv', sep='\t')
df_sid = df_sid.rename(columns=cols)[['eid', 'plate', 'well']]
df_sid.head()

,eid,plate,well
0,4219054,SMP4_0012173A,F10
1,4219069,SMP4_0012961A,G12
2,4219076,SMP4_0016227A,E04
3,4219081,SMP4_0014664A,F09
4,4219097,SMP4_0014570A,B07


In [5]:
df_snl = pd.read_csv('gs://rs-ukb/pipe-data/external/nealelab_v3_20180731/extract/sample_sets.csv', sep='\t')
df_snl.head()

,plate,well,in_european_samples
0,SMP4_0011041A,G06,True
1,SMP4_0012545A,F08,True
2,SMP4_0016507A,H11,True
3,SMP4_0011613A,E07,True
4,SMP4_0012223B,A01,True


In [6]:
df_mrg = pd.merge(df_sid, df_snl, on=['plate', 'well'])
df_mrg = df_mrg.pipe(lambda df: df[df['in_european_samples']])
sample_ids = df_mrg['eid']
len(sample_ids)

358010

In [7]:
sample_ids[:5]

0    4219054
1    4219069
2    4219076
3    4219081
4    4219102
Name: eid, dtype: int64

In [8]:
assert (ds['sample_id1'] == ds['sample_id2']).all().compute().item(0)

In [9]:
ds['sample_id'] = ds['sample_id1']

In [10]:
dss = ds.sel(samples=ds['sample_id'].isin(sample_ids))
dss

,Array,Chunk
Bytes,1.81 TB,397.03 MB
Shape,"(1261158, 358005)","(1024, 96931)"
Count,9857 Tasks,4928 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,451.50 GB,99.26 MB
Shape,"(1261158, 358005)","(1024, 96931)"
Count,9857 Tasks,4928 Chunks
Type,bool,numpy.ndarray
,Array,Chunk
